In [8]:
from copy import deepcopy

import re

def replace_multiple_newlines(text):
    # 3つ以上の連続する改行を2つの改行に置換
    return re.sub(r'\n{2,}', '\n\n', remove_spaces_from_blank_lines(text))

def remove_spaces_from_blank_lines(text):
    # 各行を処理し、スペースのみの行を空行に置き換える
    return re.sub(r'^\s+$', '', text, flags=re.MULTILINE)

def accumulator_naive_c(accm_op, variables, indent_level=1):
    indent = " " * 4 * indent_level
    
    accm_op.append("\n")
    if len(variables) == 1:
        return variables[0]

    variables_next_ = []
    variables_ = variables[:-1] if len(variables) % 2 == 1 else variables
    for i in range(0, len(variables_), 2):
        accm_op.append(f"{indent}{variables_[i]} = {variables_[i]} + {variables_[i+1]};")
        variables_next_.append(f"{variables_[i]}")
        
    if len(variables) % 2 == 1:
        variables_next_.append(f"{variables[-1]}")
        
    accumulator_naive_c(accm_op, variables_next_, indent_level = indent_level)
    
with open(f"./small_dgemv_naive_c_implementation_ver0-4_Naive-Unroll-Complete.c", "w") as p:
    p.write("#include <stddef.h>\n")
    p.write("#include <stdio.h>\n")
    p.write("#include <stdlib.h>\n")    
    p.write("#include <stdint.h>\n")
    p.write("#include <math.h>\n")
    p.write("#include <omp.h>\n")
    p.write("#include <immintrin.h>\n") 
    
    for ldx in range(2, 514, 2):

        # Declaration
        declare_x = "double " + ", ".join([f"r{ld}" for ld in range(ldx)])
        variables = [f"r{ld}" for ld in range(ldx)]

        # Initialize
        mul_add = ""
        accm = ""

        # Multiply and Add
        accm_variables = []
        for ld in range(0, len(variables), 2):
            mul_add += f"""
        r{ld+0} = a_t[{ld}+base_idx];
        r{ld+1} = a_t[{ld+1}+base_idx]; 
        r{ld+0} = x[{ld}]*r{ld+0} + x[{ld+1}]*r{ld+1};\n"""
            accm_variables.append(f"r{ld+0}")

        #　Accumulate
        accm_op = []
        accumulator_naive_c(accm_op, accm_variables, indent_level=2)
        accm_op = "\n".join(accm_op)

        # Create Function
        base = f"""
void mydgemv_ver0_4_c_{ldx}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    {declare_x};

    int64_t base_idx;
    for (int i=0; i<lda; i++){{
        base_idx = i*ldx;
    {mul_add}
    {accm_op}
        y[i]=r0;
    }}
}}
        """
        p.write(replace_multiple_newlines(base))

In [9]:
for i in range(2, 514, 2):
#     print(f"""
#         subroutine mydgemv_ver0_4_c_{i}(a_t, x, y, lda, ldx, ldy) bind(C, name='mydgemv_ver0_4_c_{i}')
#             import
#             integer(c_int64_t), value :: lda, ldx, ldy
#             real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
#         end subroutine mydgemv_ver0_4_c_{i}
#           """)
    print(f"if (ldx == {i}) sub_ptr => mydgemv_ver0_4_c_{i}")

if (ldx == 2) sub_ptr => mydgemv_ver0_4_c_2
if (ldx == 4) sub_ptr => mydgemv_ver0_4_c_4
if (ldx == 6) sub_ptr => mydgemv_ver0_4_c_6
if (ldx == 8) sub_ptr => mydgemv_ver0_4_c_8
if (ldx == 10) sub_ptr => mydgemv_ver0_4_c_10
if (ldx == 12) sub_ptr => mydgemv_ver0_4_c_12
if (ldx == 14) sub_ptr => mydgemv_ver0_4_c_14
if (ldx == 16) sub_ptr => mydgemv_ver0_4_c_16
if (ldx == 18) sub_ptr => mydgemv_ver0_4_c_18
if (ldx == 20) sub_ptr => mydgemv_ver0_4_c_20
if (ldx == 22) sub_ptr => mydgemv_ver0_4_c_22
if (ldx == 24) sub_ptr => mydgemv_ver0_4_c_24
if (ldx == 26) sub_ptr => mydgemv_ver0_4_c_26
if (ldx == 28) sub_ptr => mydgemv_ver0_4_c_28
if (ldx == 30) sub_ptr => mydgemv_ver0_4_c_30
if (ldx == 32) sub_ptr => mydgemv_ver0_4_c_32
if (ldx == 34) sub_ptr => mydgemv_ver0_4_c_34
if (ldx == 36) sub_ptr => mydgemv_ver0_4_c_36
if (ldx == 38) sub_ptr => mydgemv_ver0_4_c_38
if (ldx == 40) sub_ptr => mydgemv_ver0_4_c_40
if (ldx == 42) sub_ptr => mydgemv_ver0_4_c_42
if (ldx == 44) sub_ptr => mydgemv_ver0_4_c